# Gear Selection Experiments

## Imports

In [1]:
from pathlib import Path
from datetime import date
import os
import subprocess
from typing import List
from copy import deepcopy
from revcan.reverse_engineering.models.experiment import Experiment, Extern_Alphanumeric_Signal

## Create Necessary Experiment Files

### TODO: 
Set & check the following parameters:
- **Car Model File Name**: Specify the name of the car model file (`{model}_{vin}_{date-created}.json`).
- **Car Model Directory**: Check the directory path where the car model file is stored (Default: `../../data/car_metadata`).
- **Experiment Folder**: Set the path for the experiment folder where the results will be saved (Default: `../../data/experiments/gear_selected_{model}/{date-today}`).
- **Gears**: Set the possible gears for the car model (Default: `["P","R","N","D"]`).

In [ ]:
car_model_file_name = "model_vin_date-created.json" # TODO: Set correct car model file name
car_model_dir = "../../data/car_metadata" # TODO: Check directory of car model file
experiment_folder = "../../data/experiments/car/gear_selected/" + date.today().strftime("%Y-%m-%d") # TODO: Set directory where experiment files should be saved to
gears=["P","R","N","D"] # TODO: Check gears


car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

print(f"Car model file path: {car_model_file_path}")

#create experiments
for gear in gears:

    experiment_file_path = os.path.join(experiment_folder, gear+".json")
    experiment_name = f"Gear selected {gear}"
    experiment_description = "gear selected signal discovery"
    signal_selection ="all"
    print(f"Experiment file path: {experiment_file_path}")
    !python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{experiment_file_path}' --experiment_name '{experiment_name}' --experiment_description '{experiment_description}' --signal_selection {signal_selection}


## [Optional]: Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/gear_selected/2025-04-03_including_D2"
gears=["P","R","N","D1","D2"] # TODO: Check gears

from pathlib import Path
import os
import subprocess
from datetime import datetime

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

## 1. Measurements

For each gear 5 measurements will be taken.

### 1.1 Experiment 1: Gear P

Before executing the next Cell: Set the gear to P 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "P"+".json")}' --activate_logging true --num_samples 4

### 1.2 Experiment 2: Gear R

Before executing the next Cell: Set the gear to R 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "R"+".json")}' --activate_logging true --num_samples 4

### 1.3 Experiment 3: Gear N

Before executing the next Cell: Set the gear to N 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "N"+".json")}' --activate_logging true --num_samples 4

### 1.4 Experiment 4: Gear D

Before executing the next Cell: Set the gear to D 

In [ ]:
!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, "D"+".json")}' --activate_logging true --num_samples 4

### Optional: 1.5 Experiment X: Additional measurements for additional gears

Before executing the next Cell: Set the gear to the desired gear

In [ ]:
additional_gear = "D2" # TODO: Set name of gear

!python ../scripts_for_doip_new/05_read_data.py --experiment_file_path '{os.path.join(experiment_folder, {additional_gear}+".json")}' --activate_logging true --num_samples 5

## [Optional] Display Experiment Metadata

In [ ]:
# Display Experiment
for gear in gears:
    experiment_file = os.path.join(experiment_folder, f"{gear}.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

## Load Experiments

Load all relevant experiments needed for this analysis

In [4]:
experiment_files = {}
experiments = {}

for gear in gears:
    experiment_files[gear] = os.path.join(experiment_folder, f"{gear}.json")
    experiments[gear] = Experiment.load(experiment_files[gear])

## Add Ground truth
Add ground truth data for steering wheel angle. By experiment design these are fixed values, and are added manually in this case

In [ ]:
for gear in gears:
    experiments[gear].external_alphanumeric_measurements.append(Extern_Alphanumeric_Signal( name = "Gear", id = 1, values = []))
    values:List[str] = []
    for value in experiments[gear].measurements[0].values:
        values.append(gear)
    experiments[gear].external_alphanumeric_measurements[0].name = "Gear"
    experiments[gear].external_alphanumeric_measurements[0].values = deepcopy(values)
    print(f"Experiment gear {gear}: {len(experiments[gear].external_alphanumeric_measurements[0].values)} ground truth values")

## Concat all experiments to combined_experiment

In [ ]:
combined_experiment = deepcopy(experiments[gears[0]])
combined_experiment.name = f"Non constant signals for gear experiment including d2"
combined_experiment.description = f"Non constant signals for gear experiment with {len(gears)} different gears. Gears: {gears}"

for gear in gears[1:]:
    combined_experiment.experiment_runtime_seconds += experiments[gear].experiment_runtime_seconds
    for measurement in experiments[gear].measurements:
        for x in combined_experiment.measurements:
            if x.serverid == measurement.serverid:
                if x.did == measurement.did:
                    x.values.extend(measurement.values)
                    break
    combined_experiment.external_alphanumeric_measurements[0].values.extend(experiments[gear].external_alphanumeric_measurements[0].values)


### Filter

Keep only non constant signals

In [ ]:
number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements before filtering", )
print("Applaying remove constant values filter")

combined_experiment.keep_non_constant_signals(combined_experiment)

number_measurements = len(combined_experiment.measurements)
print(f"Experiment contains {number_measurements} measurements after filtering", )

## Save combined experiment file

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"experiment_non_constant_signals.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"

## [OPTIONAL] Remove all previously found constant signals from individual experiments

In [ ]:
for gear in gears:
    experiments[gear].keep_signals_by_list(signals_list=combined_experiment.measurements)
    experiment_file = os.path.join(experiment_folder, f"{gear}.json")
    experiments[gear].save(f"{experiment_file}")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"